In [4]:
import os
import sys

import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from pathlib import Path
import matplotlib.pyplot as plt


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from data_management.indicator_manager import IndicatorManager
from visualization.chart_manager import ChartManager
from data_management.preprocessor import DataPreprocessor

processor = DataPreprocessor()
dataset_manager = DatasetManager()
chart_manager = ChartManager()

In [5]:
eur_1min_spirce = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/1min/EUR_USD.parquet'
df = pd.read_parquet(eur_1min_spirce)
indicator_manager = IndicatorManager()
df_with_indicators = indicator_manager.calculate_indicators(df, indicator_timeframe='5T')
# df_with_indicators.dropna(inplace=True)

# print(df_with_indicators)
df_norm = processor.normalize_simple(df=df_with_indicators)
df_norm.to_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/EUR_USD_5T_indics_norm2.parquet', compression=None)
df_norm

# df_with_indicators = df_with_indicators['2024-11-27 08:00':'2024-11-27 09:00']
# plt.figure(figsize=(15, 6))
# plt.plot(df_with_indicators.index, df_with_indicators['macd_hist'])
# plt.title('MACD Histogram')
# plt.xlabel('Date')
# plt.ylabel('MACD Histogram Value')
# plt.grid(True)
# plt.show()



/Users/floriankockler/Code/GitHub.nosync/ai6-gcp-bot/forex_trading_system/data_management/indicator_manager.py:69: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df.resample(indicator_timeframe).agg({


Added indicators: ['sma_20', 'sma_50', 'rsi', 'macd', 'macd_signal', 'macd_hist', 'bb_upper', 'bb_middle', 'bb_lower', 'bb_bandwidth', 'bb_percent', 'atr', 'plus_di', 'minus_di', 'adx', 'senkou_span_a', 'senkou_span_b', 'tenkan_sen', 'kijun_sen']
Original shape: (8517438, 4), Final shape: (8517438, 23)
Normalized sma_20 by price
Normalized sma_50 by price
Normalized rsi by percentage
Normalized macd by price
Normalized macd_signal by price
Normalized macd_hist by price
Normalized bb_upper by price
Normalized bb_middle by price
Normalized bb_lower by price
Normalized bb_bandwidth by price
Normalized bb_percent by percentage
Normalized atr by price
Normalized plus_di by percentage
Normalized minus_di by percentage
Normalized adx by percentage
Normalized senkou_span_a by price
Normalized senkou_span_b by price
Normalized tenkan_sen by price
Normalized kijun_sen by price


,open,high,low,close,sma_20,sma_50,rsi,macd,macd_signal,macd_hist,...,bb_percent,atr,plus_di,minus_di,adx,senkou_span_a,senkou_span_b,tenkan_sen,kijun_sen,price_norm
timestamp,,,,,,,,,,,,,,,,,,,,,
2001-01-03 05:30:00+00:00,0.95060,0.95060,0.95060,0.95060,0.999911,0.999745,0.574432,0.000085,0.000084,0.000001,...,0.709637,0.000103,0.365211,0.195930,0.289345,0.999632,0.999474,1.000000,0.999737,1.000000
2001-01-03 05:32:00+00:00,0.95060,0.95060,0.95060,0.95060,0.999911,0.999745,0.574432,0.000085,0.000084,0.000001,...,0.709637,0.000103,0.365211,0.195930,0.289345,0.999632,0.999474,1.000000,0.999737,1.000000
2001-01-03 05:33:00+00:00,0.95060,0.95060,0.95060,0.95060,0.999911,0.999745,0.574432,0.000085,0.000084,0.000001,...,0.709637,0.000103,0.365211,0.195930,0.289345,0.999632,0.999474,1.000000,0.999737,1.000000
2001-01-03 05:34:00+00:00,0.95060,0.95060,0.95060,0.95060,0.999911,0.999745,0.574432,0.000085,0.000084,0.000001,...,0.709637,0.000103,0.365211,0.195930,0.289345,0.999632,0.999474,1.000000,0.999737,1.000000
2001-01-03 05:35:00+00:00,0.95060,0.95060,0.95060,0.95060,0.999926,0.999743,0.574432,0.000082,0.000084,-0.000001,...,0.000000,0.000096,0.365211,0.195930,0.290225,0.999684,0.999474,1.000000,0.999842,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-28 10:13:00+00:00,1.05362,1.05369,1.05351,1.05360,1.000225,1.000488,0.462109,-0.000223,-0.000197,-0.000026,...,0.350479,0.000567,0.204235,0.173221,0.124934,1.000517,1.000650,0.999900,1.000669,1.108353
2024-11-28 10:14:00+00:00,1.05358,1.05365,1.05338,1.05356,1.000263,1.000526,0.462109,-0.000223,-0.000197,-0.000026,...,0.350479,0.000567,0.204235,0.173221,0.124934,1.000555,1.000688,0.999938,1.000707,1.108311
2024-11-28 10:15:00+00:00,1.05357,1.05360,1.05338,1.05340,1.000395,1.000659,0.458979,-0.000206,-0.000199,-0.000007,...,0.352124,0.000547,0.196393,0.166570,0.121879,1.000600,1.000840,1.000090,1.000883,1.108142


In [4]:

# Check for inf/-inf values in each column
inf_counts = df_norm.isin([np.inf, -np.inf]).sum()
print("Number of infinite values in each column:")
print(inf_counts[inf_counts > 0])  # Only show columns with inf values


Number of infinite values in each column:
Series([], dtype: int64)


In [ ]:
df_with_indicators.isna().sum()

In [6]:
import pyarrow.parquet as pq


df_norm_5min = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/EUR_USD_5T_indics_norm2.parquet'
import pandas as pd

df = pd.read_parquet(df_norm_5min)
df

OSError: Couldn't deserialize thrift: TProtocolException: Invalid data
Deserializing page header failed.


In [ ]:
df_norm.to_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/EUR_USD_5T_indics_norm.parquet')

In [ ]:
start_time = pd.Timestamp('2024-11-27 08:00').tz_localize('UTC')
end_time = pd.Timestamp('2024-11-27 09:00').tz_localize('UTC')

chart_manager.chart(df_with_indicators, start_time, end_time)

EUR norm Hstack

In [1]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager

pair = "EUR_USD"

df_norm_5min = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/EUR_USD_5T_indics_norm.parquet'

df = pd.read_parquet(df_norm_5min)

dataset_manager = DatasetManager()
train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)



saving_path = f'./logs/28nov/norm_5min/'
os.makedirs(saving_path, exist_ok=True)

def make_train_env():
    env = ForexTradingEnv(
        df=train_df,
        pair='EUR_USD',

    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True)
    return env

def make_eval_env():
    env = ForexTradingEnv(

        df=val_df,
        pair='EUR_USD',
        # resample_interval='1h'
    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=False)
    env.training = False
    return env

train_env = make_train_env()
eval_env = make_eval_env()
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=saving_path,
    log_path=saving_path,
    eval_freq=100_000,  # Adjust as needed
    n_eval_episodes=5,
    deterministic=True,
    render=False
)

model = PPO(
    'MlpPolicy',
    train_env,
    verbose=0,
    tensorboard_log=f'{saving_path}tensorboard/',
)

model.learn(
    total_timesteps=20_000_000,  # Adjust as needed
    callback=eval_callback
)

model.save(f'{saving_path}best_model.zip')
train_env.save(f'{saving_path}vec_normalize.pkl')

OSError: Corrupt snappy compressed data.

In [2]:
df_norm_5min = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/EUR_USD_5T_indics_norm.parquet'

df = pd.read_parquet(df_norm_5min)
dataset_manager = DatasetManager()
train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)
train_df['bb_percent']

Dataset split sizes:
Training: 5961956 samples (70.0%)
Validation: 1277562 samples (15.0%)
Test: 1277563 samples (15.0%)


timestamp
2001-01-03 05:30:00+00:00    0.709637
2001-01-03 05:32:00+00:00    0.709637
2001-01-03 05:33:00+00:00    0.709637
2001-01-03 05:34:00+00:00    0.709637
2001-01-03 05:35:00+00:00         inf
                               ...   
2017-12-13 04:50:00+00:00    0.821499
2017-12-13 04:51:00+00:00    0.821499
2017-12-13 04:52:00+00:00    0.821499
2017-12-13 04:53:00+00:00    0.821499
2017-12-13 04:54:00+00:00    0.821499
Name: bb_percent, Length: 5961956, dtype: float64

In [1]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager

pair = "EUR_USD"

df_norm_5min = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/EUR_USD_5T_indics_norm.parquet'

df = pd.read_parquet(df_norm_5min)
df.columns

Index(['open', 'high', 'low', 'close', 'sma_20', 'sma_50', 'rsi', 'macd',
       'macd_signal', 'macd_hist', 'bb_upper', 'bb_middle', 'bb_lower',
       'bb_bandwidth', 'bb_percent', 'atr', 'plus_di', 'minus_di', 'adx',
       'senkou_span_a', 'senkou_span_b', 'tenkan_sen', 'kijun_sen',
       'price_norm'],
      dtype='object')

In [ ]:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pathlib import Path
import os, sys
output_dir= "/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/normalized/"
Path(output_dir)
# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


from data_management.preprocessor import DataPreprocessor



processor = DataPreprocessor()



eur_norm = processor.normalize_simple(df=eur)
eur_norm


In [ ]:
eur_norm.to_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/normalized/EUR_USD.parquet')